In [1]:
%%capture

# import libraries
import csv
import random
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from geopy.distance import geodesic
from shapely import LineString
from helper import find_midpoint, compute_midpoints

# load data into dataframes
df_dam = pd.read_csv("data/df_dam.csv")
df_neg = pd.read_csv("data/df_neg.csv")

In [8]:
df_points = compute_midpoints(df_dam)
df_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       322 non-null    object 
 1   longitude  322 non-null    float64
 2   latitude   322 non-null    float64
dtypes: float64(2), object(1)
memory usage: 7.7+ KB


In [9]:
df_polygons = pd.read_csv("data/colorado_jan15.csv", encoding='ISO-8859-1')
df_polygons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 58 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Landsat Product Identifier L2  320 non-null    object 
 1   Landsat Product Identifier L1  320 non-null    object 
 2   Landsat Scene Identifier       320 non-null    object 
 3   Date Acquired                  320 non-null    object 
 4   Collection Category            320 non-null    object 
 5   Collection Number              320 non-null    int64  
 6   WRS Path                       320 non-null    int64  
 7   WRS Row                        320 non-null    int64  
 8   Target WRS Path                320 non-null    int64  
 9   Target WRS Row                 320 non-null    int64  
 10  Nadir/Off Nadir                320 non-null    object 
 11  Roll Angle                     320 non-null    float64
 12  Date Product Generated L2      320 non-null    obj

In [10]:
import pandas as pd
from shapely.geometry import Point, Polygon

polygons = []
for index, row in df_polygons.iterrows():
    polygon = Polygon([
        (row['Corner Upper Left Longitude'], row['Corner Upper Left Latitude']),
        (row['Corner Upper Right Longitude'], row['Corner Upper Right Latitude']),
        (row['Corner Lower Right Longitude'], row['Corner Lower Right Latitude']),
        (row['Corner Lower Left Longitude'], row['Corner Lower Left Latitude']),
        (row['Corner Upper Left Longitude'], row['Corner Upper Left Latitude'])
    ])
    polygons.append((polygon, row['Landsat Product Identifier L2']))

def find_polygon(point):
    for polygon, identifier in polygons:
        if polygon.contains(point):
            return identifier
    return None

df_points['Polygon Identifier'] = df_points.apply(
    lambda row: find_polygon(Point(row['longitude'], row['latitude'])), axis=1
)

print(df_points)

             name   longitude   latitude  \
0      Colo_Dam_1 -106.596723  40.897068   
1     Colo_Dam_10 -106.642626  40.906016   
2    Colo_Dam_100 -106.615058  40.974412   
3    Colo_Dam_101 -106.618137  40.971023   
4    Colo_Dam_102 -106.614194  40.966955   
..            ...         ...        ...   
317   Colo_Dam_95 -106.615994  40.974218   
318   Colo_Dam_96 -106.615736  40.974008   
319   Colo_Dam_97 -106.616207  40.973742   
320   Colo_Dam_98 -106.616327  40.974217   
321   Colo_Dam_99 -106.615415  40.973989   

                           Polygon Identifier  
0    LC08_L2SP_035031_20160104_20200907_02_T1  
1    LC08_L2SP_035031_20160104_20200907_02_T1  
2    LC08_L2SP_035031_20160104_20200907_02_T1  
3    LC08_L2SP_035031_20160104_20200907_02_T1  
4    LC08_L2SP_035031_20160104_20200907_02_T1  
..                                        ...  
317  LC08_L2SP_035031_20160104_20200907_02_T1  
318  LC08_L2SP_035031_20160104_20200907_02_T1  
319  LC08_L2SP_035031_20160104_20200907

In [11]:
df_dam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4957 entries, 0 to 4956
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Longitude     4957 non-null   float64
 1   Latitude      4957 non-null   float64
 2   Name          4957 non-null   object 
 3   system:index  4957 non-null   object 
 4   altitudeMo    0 non-null      float64
 5   begin         0 non-null      float64
 6   descriptio    0 non-null      float64
 7   end           0 non-null      float64
 8   extrude       4957 non-null   int64  
 9   icon          0 non-null      float64
 10  layer         4957 non-null   object 
 11  path          4957 non-null   object 
 12  tessellate    4957 non-null   int64  
 13  timestamp     0 non-null      float64
 14  visibility    4957 non-null   int64  
dtypes: float64(8), int64(3), object(4)
memory usage: 581.0+ KB


In [12]:
df_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                322 non-null    object 
 1   longitude           322 non-null    float64
 2   latitude            322 non-null    float64
 3   Polygon Identifier  322 non-null    object 
dtypes: float64(2), object(2)
memory usage: 10.2+ KB


In [14]:
import pandas as pd
from shapely.geometry import Point, Polygon, LineString

polygons = []
for index, row in df_polygons.iterrows():
    polygon = Polygon([
        (row['Corner Upper Left Longitude'], row['Corner Upper Left Latitude']),
        (row['Corner Upper Right Longitude'], row['Corner Upper Right Latitude']),
        (row['Corner Lower Right Longitude'], row['Corner Lower Right Latitude']),
        (row['Corner Lower Left Longitude'], row['Corner Lower Left Latitude']),
        (row['Corner Upper Left Longitude'], row['Corner Upper Left Latitude'])
    ])
    polygons.append((polygon, row['Landsat Product Identifier L2']))

linestrings = df_dam.groupby('Name').apply(
    lambda group: LineString(zip(group['Longitude'], group['Latitude']))
).reset_index()
linestrings.columns = ['Name', 'LineString']

df_points = df_points.rename(columns={'name': 'Name'})
linestrings = linestrings.merge(df_points[['Name', 'Polygon Identifier']], on='Name', how='left')

def is_on_boundary(linestring, polygon_identifier):
    for polygon, identifier in polygons:
        if identifier == polygon_identifier:
            return linestring.intersects(polygon.boundary)
    return False

linestrings['OnBoundary'] = linestrings.apply(
    lambda row: is_on_boundary(row['LineString'], row['Polygon Identifier']), axis=1
)

print(linestrings)

             Name                                         LineString  \
0      Colo_Dam_1  LINESTRING (-106.5970268558473 40.896725575497...   
1     Colo_Dam_10  LINESTRING (-106.6426122459167 40.905973751082...   
2    Colo_Dam_100  LINESTRING (-106.6150550014064 40.974287179221...   
3    Colo_Dam_101  LINESTRING (-106.6180693537185 40.970925017027...   
4    Colo_Dam_102  LINESTRING (-106.6142924951262 40.966911826079...   
..            ...                                                ...   
317   Colo_Dam_95  LINESTRING (-106.6161563993666 40.974086519674...   
318   Colo_Dam_96  LINESTRING (-106.6157104892612 40.973573723052...   
319   Colo_Dam_97  LINESTRING (-106.6160672173455 40.973297258787...   
320   Colo_Dam_98  LINESTRING (-106.6163303043076 40.974082060573...   
321   Colo_Dam_99  LINESTRING (-106.6153849748843 40.973743168892...   

                           Polygon Identifier  OnBoundary  
0    LC08_L2SP_035031_20160104_20200907_02_T1       False  
1    LC08_L2SP_

In [17]:
boundary_counts = linestrings['OnBoundary'].value_counts()

print(boundary_counts)

OnBoundary
False    322
Name: count, dtype: int64
